# Testing MHW Systems

In [94]:
# imports
from importlib import reload
import numpy as np
import os
from matplotlib import pyplot as plt

from datetime import date

import pandas
import sqlalchemy
import iris

from oceanpy.sst import io as sst_io
from mhw_analysis.systems import cube as mhw_cube
from mhw_analysis.systems import build as mhw_build

# Load a few things for guidance

In [4]:
SST = sst_io.load_noaa((1, 1, 2014))

In [5]:
SST

<iris 'Cube' of Daily Sea Surface Temperature / (degC) (latitude: 720; longitude: 1440)>

In [7]:
lats = SST.coord('latitude').points
lats[lats>40.]

array([40.125, 40.375, 40.625, 40.875, 41.125, 41.375, 41.625, 41.875,
       42.125, 42.375, 42.625, 42.875, 43.125, 43.375, 43.625, 43.875,
       44.125, 44.375, 44.625, 44.875, 45.125, 45.375, 45.625, 45.875,
       46.125, 46.375, 46.625, 46.875, 47.125, 47.375, 47.625, 47.875,
       48.125, 48.375, 48.625, 48.875, 49.125, 49.375, 49.625, 49.875,
       50.125, 50.375, 50.625, 50.875, 51.125, 51.375, 51.625, 51.875,
       52.125, 52.375, 52.625, 52.875, 53.125, 53.375, 53.625, 53.875,
       54.125, 54.375, 54.625, 54.875, 55.125, 55.375, 55.625, 55.875,
       56.125, 56.375, 56.625, 56.875, 57.125, 57.375, 57.625, 57.875,
       58.125, 58.375, 58.625, 58.875, 59.125, 59.375, 59.625, 59.875,
       60.125, 60.375, 60.625, 60.875, 61.125, 61.375, 61.625, 61.875,
       62.125, 62.375, 62.625, 62.875, 63.125, 63.375, 63.625, 63.875,
       64.125, 64.375, 64.625, 64.875, 65.125, 65.375, 65.625, 65.875,
       66.125, 66.375, 66.625, 66.875, 67.125, 67.375, 67.625, 67.875,
      

In [11]:
lons = SST.coord('longitude').points
lons[(lons > 200) & (lons < 230)]

array([200.125, 200.375, 200.625, 200.875, 201.125, 201.375, 201.625,
       201.875, 202.125, 202.375, 202.625, 202.875, 203.125, 203.375,
       203.625, 203.875, 204.125, 204.375, 204.625, 204.875, 205.125,
       205.375, 205.625, 205.875, 206.125, 206.375, 206.625, 206.875,
       207.125, 207.375, 207.625, 207.875, 208.125, 208.375, 208.625,
       208.875, 209.125, 209.375, 209.625, 209.875, 210.125, 210.375,
       210.625, 210.875, 211.125, 211.375, 211.625, 211.875, 212.125,
       212.375, 212.625, 212.875, 213.125, 213.375, 213.625, 213.875,
       214.125, 214.375, 214.625, 214.875, 215.125, 215.375, 215.625,
       215.875, 216.125, 216.375, 216.625, 216.875, 217.125, 217.375,
       217.625, 217.875, 218.125, 218.375, 218.625, 218.875, 219.125,
       219.375, 219.625, 219.875, 220.125, 220.375, 220.625, 220.875,
       221.125, 221.375, 221.625, 221.875, 222.125, 222.375, 222.625,
       222.875, 223.125, 223.375, 223.625, 223.875, 224.125, 224.375,
       224.625, 224.

# Generate a faux pandas MHW Event table

## Pacific

### Locations

In [12]:
pac_lons = lons[(lons > 200) & (lons < 230)]
pac_lons.size

120

In [13]:
pac_lats = lats[(lats>40.) & (lats < 60.)]
pac_lats.size

80

In [23]:
pac_XX, pac_YY = np.meshgrid(pac_lons, pac_lats)

In [25]:
pac_XX.shape

(80, 120)

### Times

In [67]:
pac_date_end = date(2013, 12, 31)
pac_date_end.toordinal()

735233

In [27]:
rand_dur = np.random.randint(5,100, pac_lats.size*pac_lons.size).reshape((80,120))
rand_dur

array([[81, 33, 13, ...,  7, 98,  6],
       [68, 14, 90, ..., 33, 26,  9],
       [24, 86, 48, ..., 44, 55, 89],
       ...,
       [ 9, 64, 17, ..., 75, 58, 16],
       [42, 30, 24, ..., 91, 79, 79],
       [31, 72, 43, ..., 27, 10, 74]])

In [28]:
pac_start = pac_date_end.toordinal() - rand_dur

In [30]:
pac_start.shape

(80, 120)

### Random categories

In [79]:
ran_cat = np.random.randint(0,4, pac_lats.size*pac_lons.size).reshape((80,120))

### Build em

In [80]:
pac_mhw_events = dict(lat=[], lon=[], duration=[], time_start=[], category=[])
for ii in range(pac_start.shape[0]):
    for jj in range(pac_start.shape[1]):
        # Fill it up
        pac_mhw_events['lat'].append(pac_YY[ii,jj])
        pac_mhw_events['lon'].append(pac_XX[ii,jj])
        pac_mhw_events['duration'].append(rand_dur[ii,jj])
        pac_mhw_events['time_start'].append(pac_start[ii,jj])
        pac_mhw_events['category'].append(ran_cat[ii,jj])

In [81]:
pac_tbl = pandas.DataFrame(pac_mhw_events)

In [82]:
pac_tbl.head()

,lat,lon,duration,time_start,category
0,40.125,200.125,81,735152,2
1,40.125,200.375,33,735200,1
2,40.125,200.625,13,735220,0
3,40.125,200.875,32,735201,3
4,40.125,201.125,12,735221,0


## Atlantic

In [41]:
atl_lons = lons[(lons > (360-70)) & (lons < (360-20))]
atl_lats = lats[(lats > 10) & (lats < 50)]

In [48]:
atl_XX, atl_YY = np.meshgrid(atl_lons, atl_lats)
atl_XX = atl_XX.flatten()
atl_YY = atl_YY.flatten()
atl_XX.shape

(32000,)

In [45]:
nrand_atl = 500

In [56]:
pos_rand = np.random.choice(np.arange(atl_XX.size), nrand_atl, replace=False)
pos_rand[0:5]

array([  510, 10455,  5809, 26731, 22621])

### Time

In [49]:
atl_dur = 7

In [51]:
atl_start = pac_date_end.toordinal() - np.random.randint(10,100, nrand_atl)

In [83]:
atl_cat = np.random.randint(0,4, nrand_atl)

### Build em

In [84]:
atl_mhw_events = dict(lat=[], lon=[], duration=[], time_start=[], category=[])
for ii in range(nrand_atl):
    # Fill it up
    atl_mhw_events['lat'].append(atl_YY[pos_rand[ii]])
    atl_mhw_events['lon'].append(atl_XX[pos_rand[ii]])
    atl_mhw_events['duration'].append(atl_dur)
    atl_mhw_events['time_start'].append(atl_start[ii])
    atl_mhw_events['category'].append(atl_cat[ii])

In [85]:
atl_tbl = pandas.DataFrame(atl_mhw_events)

In [86]:
atl_tbl.head()

,lat,lon,duration,time_start,category
0,10.625,317.625,7,735193,2
1,23.125,303.875,7,735180,0
2,17.375,292.375,7,735171,1
3,43.375,322.875,7,735172,3
4,38.375,295.375,7,735187,1


## Combine

In [87]:
mhw_tbl = pandas.concat([pac_tbl, atl_tbl])
mhw_tbl.head()

,lat,lon,duration,time_start,category
0,40.125,200.125,81,735152,2
1,40.125,200.375,33,735200,1
2,40.125,200.625,13,735220,0
3,40.125,200.875,32,735201,3
4,40.125,201.125,12,735221,0


# Cube

In [88]:
reload(mhw_cube)
cube = mhw_cube.build_cube('test_cube.npz', mhw_events=mhw_tbl, dmy_start=(2013,1,1), dmy_end=(2013,12,31))

Events are loaded
kk = 0
Wrote: test_cube.npz


In [92]:
np.max(cube), cube.dtype

(4, dtype('int8'))

# Systems

In [95]:
reload(mhw_build)
mhw_sys, mhw_mask = mhw_build.

<module 'mhw_analysis.systems.build' from '/home/xavier/Projects/Oceanography/Python/mhw_analysis/mhw_analysis/systems/build.py'>